In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time
import json


In [2]:
options = webdriver.ChromeOptions()
chrome_options = Options()
#options.headless = True

In [3]:
rating_dict = {'it was amazing': 5,
               'really liked it': 4,
              'liked it': 3,
              'it was ok': 2,
              'did not like it': 1
              }


def convert_text_to_number_rating(rating_str):
    return int(rating_dict[rating_str])
    

In [4]:
url = 'https://www.goodreads.com/book/show/3.Harry_Potter_and_the_Sorcerer_s_Stone'

url2 = 'https://www.goodreads.com/book/show/14296652-the-unofficial-narnia-cookbook'

In [5]:
def get_first_three_info():
    info_dict = {}
    info_legnth = len(browser.find_elements_by_class_name("infoBoxRowTitle"))
    
    if info_legnth >= 3 :
        info_legnth = 3
        
    
    for i in range(0, info_legnth):
        key = browser.find_elements_by_class_name("infoBoxRowTitle")[i].text
        val = browser.find_elements_by_class_name("infoBoxRowItem")[i].text
        
        info_dict[key] = val
        
    if 'ISBN' in info_dict:
        return info_dict['ISBN']
    
    else:
        return 'empty'

In [6]:
def book_page(book_url):
    
    
    
    browser.get(book_url)
    
    book_title = browser.find_element_by_id('bookTitle').text
    
    more_info_button = browser.find_element_by_id("bookDataBoxShow")
    more_info_button.click()
    
    isbn = get_first_three_info().split(' ')[0]
    
    
#    book_dict = get_multiple_reviews_by_page(book_title, isbn)
    book_review_count = get_multiple_reviews_by_page(book_title, isbn)
    browser.close()
    browser.quit()
    
    
    return book_review_count
    
        
    

In [7]:
def get_reviews_for_book(book_title, isbn):
    
    count = 0
    rating_dict_list = []
    attempts = 0
    
    while attempts <5:
        
        try:
            
            time.sleep(2)
            reviews_list = browser.find_elements_by_class_name('friendReviews')
            break
            
        except:
            
            attempts +=1
            
            print("This is attempt number: {}".format(attempts))
            
            if attempts == 5:
                
                return rating_dict_list
    
    
    for i in reviews_list:
        
        if 'rated' in i.text.split('\n')[1]:
            
            rating = i.text.split('\n')[2]
            rating = convert_text_to_number_rating(rating)
            user_name = i.find_element_by_class_name('user').text
            user_id = i.find_element_by_class_name('user').get_attribute("href").split('/')[-1].split('-')[0]
            
            try:
                likes = i.find_element_by_xpath('.//span[@class = "likesCount"]').text.split(' ')[0]
            except:
                try:
                    likes = i.find_element_by_class_name('likesCount').text.split(' ')[0]
                except:
                    likes = 0
            
            count +=1
            
            ind_book_dict = {'user_id': user_id, 'user_name': user_name, 'rating': rating, 'isbn': isbn, 'title': book_title, 'likes': int(likes)}
            
            with open("book_reviews.txt", "a") as write_file:
                write_file.write(str(ind_book_dict) + ', ') 
            
#            rating_dict_list.append({'user_id': user_id, 'user_name': user_name, 'rating': rating, 'isbn': isbn, 'title': book_title, 'likes': int(likes)})
            
            

#    return rating_dict_list
    return count

In [8]:
def get_multiple_reviews_by_page(book_title, isbn):
    
    ind_review_count = 0
    all_reviews = []
    number = 1
    
    page_total = len(browser.find_element_by_xpath('//*[@id="reviews"]/div[4]/div').text.split(' ')[2:-2])
    
    for i in range(0, page_total):
        
        number += 1
        
#        ind_review_dict = get_reviews_for_book(book_title, isbn)
#        all_reviews += ind_review_dict


        ind_review_count += get_reviews_for_book(book_title, isbn)
        
        print("Total reviews added for {}: {}".format(book_title, ind_review_count))
        
        if i < page_total-1:
        
            time.sleep(2)
            next_page(str(number))
            time.sleep(2)
            browser.execute_script("window.scrollTo(0, 12000);")
    
    return ind_review_count
        
        

In [9]:
def next_page(page_number):
    
    attempts = 0
    
    while attempts <15:
        
        try:
            page_link = browser.find_element_by_link_text(page_number)
            page_link.click()
            break
        
        except:
            attempts +=1
            print("This is attempt to reach page number {}: {} time(s)".format(page_number, attempts))
            time.sleep(2)
            browser.execute_script("window.scrollTo(0, 200);")

In [10]:
book_df = pd.read_csv('book_info.csv')
len(book_df[103:].url)

417

In [11]:
book_df.loc[103:]

,Unnamed: 0,author,mean_rating,num_ratings,num_reviews,title,url
103,103,Kathryn Stockett,4.46,"1,874,160 ratings","81,826 reviews",The Help,https://www.goodreads.com/book/show/4667024-th...
104,104,Kristin Hannah,4.56,"508,107 ratings","51,843 reviews",The Nightingale,https://www.goodreads.com/book/show/21853621-t...
105,105,Andy Weir,4.40,"681,979 ratings","68,222 reviews",The Martian,https://www.goodreads.com/book/show/18007564-t...
106,106,Gillian Flynn,4.06,"1,965,928 ratings","127,597 reviews",Gone Girl,https://www.goodreads.com/book/show/12837725-g...
107,107,Neil Gaiman,4.00,"415,327 ratings","43,203 reviews",The Ocean at the End of the Lane,https://www.goodreads.com/book/show/15783514-t...
108,108,Suzanne Collins,4.29,"2,193,652 ratings","91,826 reviews",Catching Fire,https://www.goodreads.com/book/show/6148028-ca...
109,109,John Green,4.24,"3,026,206 ratings","149,343 reviews",The Fault in Our Stars,https://www.goodreads.com/book/show/11870085-t...
110,110,Stephen King,4.31,"365,150 ratings","34,456 reviews",11/22/63,https://www.goodreads.com/book/show/10644930-1...
111,111,Sarah J. Maas,4.58,"178,067 ratings","18,503 reviews",Queen of Shadows,https://www.goodreads.com/book/show/18006496-q...
112,112,George R.R. Martin,4.32,"495,558 ratings","21,323 reviews",A Dance with Dragons,https://www.goodreads.com/book/show/13422727-a...


In [ ]:
#all_book_reviews = pd.DataFrame(columns= ['user_id', 'user_name', 'rating', 'isbn', 'title', 'likes'])
#all_book_reviews.head()

In [16]:
book_df.loc[book_df.title == 'The Fates Divide']

,Unnamed: 0,author,mean_rating,num_ratings,num_reviews,title,url
452,452,Veronica Roth,3.93,"13,506 ratings","1,483 reviews",The Fates Divide,https://www.goodreads.com/book/show/35820633-t...


In [17]:
book_df[452:].url.unique

<bound method Series.unique of 452    https://www.goodreads.com/book/show/35820633-t...
453    https://www.goodreads.com/book/show/34726469-s...
454    https://www.goodreads.com/book/show/25446343-m...
455    https://www.goodreads.com/book/show/39348113-s...
456    https://www.goodreads.com/book/show/34275232-t...
457    https://www.goodreads.com/book/show/36329818-l...
458    https://www.goodreads.com/book/show/29456569-l...
459    https://www.goodreads.com/book/show/35495083-s...
460    https://www.goodreads.com/book/show/28006096-t...
461    https://www.goodreads.com/book/show/35403058-c...
462    https://www.goodreads.com/book/show/33784323-t...
463    https://www.goodreads.com/book/show/36222611-a...
464    https://www.goodreads.com/book/show/36086513-a...
465    https://www.goodreads.com/book/show/35959634-g...
466    https://www.goodreads.com/book/show/36644148-t...
467    https://www.goodreads.com/book/show/36546966-w...
468    https://www.goodreads.com/book/show/33004208-t...


In [18]:
book_df[452:].url.nunique()

48

In [19]:
for i in book_df[452:].url.unique():

    browser = webdriver.Chrome('./chromedriver', options=options)
    browser.header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'}
    bookreviews = book_page(i)
    
    #all_book_reviews = all_book_reviews.append(pd.DataFrame(bookreviews))
    #all_book_reviews.to_csv('book_reviews.csv', index=False)
    
    

Total reviews added for The Fates Divide: 24
Total reviews added for The Fates Divide: 50
Total reviews added for The Fates Divide: 79
Total reviews added for The Fates Divide: 108
Total reviews added for The Fates Divide: 137
Total reviews added for The Fates Divide: 167
Total reviews added for The Fates Divide: 196
Total reviews added for The Fates Divide: 225
Total reviews added for The Fates Divide: 255
Total reviews added for The Fates Divide: 284
Total reviews added for Sky in the Deep: 30
Total reviews added for Sky in the Deep: 58
Total reviews added for Sky in the Deep: 85
Total reviews added for Sky in the Deep: 113
Total reviews added for Sky in the Deep: 141
Total reviews added for Sky in the Deep: 170
Total reviews added for Sky in the Deep: 200
Total reviews added for Sky in the Deep: 227
Total reviews added for Sky in the Deep: 256
Total reviews added for Sky in the Deep: 286
Total reviews added for Muse of Nightmares: 26
Total reviews added for Muse of Nightmares: 50
Th

Total reviews added for Ghost Boys: 176
Total reviews added for Ghost Boys: 206
Total reviews added for Ghost Boys: 235
Total reviews added for Ghost Boys: 265
Total reviews added for Ghost Boys: 295
Total reviews added for The Penderwicks at Last: 25
Total reviews added for The Penderwicks at Last: 52
Total reviews added for The Penderwicks at Last: 82
Total reviews added for The Penderwicks at Last: 110
Total reviews added for The Penderwicks at Last: 138
Total reviews added for The Penderwicks at Last: 168
Total reviews added for The Penderwicks at Last: 198
Total reviews added for The Penderwicks at Last: 228
Total reviews added for The Penderwicks at Last: 258
Total reviews added for The Penderwicks at Last: 288
Total reviews added for Willa of the Wood: 28
Total reviews added for Willa of the Wood: 57
Total reviews added for Willa of the Wood: 85
Total reviews added for Willa of the Wood: 115
Total reviews added for Willa of the Wood: 145
Total reviews added for Willa of the Wood

This is attempt to reach page number 11: 13 time(s)
This is attempt to reach page number 11: 14 time(s)
This is attempt to reach page number 11: 15 time(s)
Total reviews added for The Unforgettable Guinevere St. Clair: 329
This is attempt to reach page number 12: 1 time(s)
This is attempt to reach page number 12: 2 time(s)
This is attempt to reach page number 12: 3 time(s)
This is attempt to reach page number 12: 4 time(s)
This is attempt to reach page number 12: 5 time(s)
This is attempt to reach page number 12: 6 time(s)
This is attempt to reach page number 12: 7 time(s)
This is attempt to reach page number 12: 8 time(s)
This is attempt to reach page number 12: 9 time(s)
This is attempt to reach page number 12: 10 time(s)
This is attempt to reach page number 12: 11 time(s)
This is attempt to reach page number 12: 12 time(s)
This is attempt to reach page number 12: 13 time(s)
This is attempt to reach page number 12: 14 time(s)
This is attempt to reach page number 12: 15 time(s)
Total 

This is attempt to reach page number 8: 13 time(s)
This is attempt to reach page number 8: 14 time(s)
This is attempt to reach page number 8: 15 time(s)
Total reviews added for Hello Lighthouse: 59
This is attempt to reach page number 9: 1 time(s)
This is attempt to reach page number 9: 2 time(s)
This is attempt to reach page number 9: 3 time(s)
This is attempt to reach page number 9: 4 time(s)
This is attempt to reach page number 9: 5 time(s)
This is attempt to reach page number 9: 6 time(s)
This is attempt to reach page number 9: 7 time(s)
This is attempt to reach page number 9: 8 time(s)
This is attempt to reach page number 9: 9 time(s)
This is attempt to reach page number 9: 10 time(s)
This is attempt to reach page number 9: 11 time(s)
This is attempt to reach page number 9: 12 time(s)
This is attempt to reach page number 9: 13 time(s)
This is attempt to reach page number 9: 14 time(s)
This is attempt to reach page number 9: 15 time(s)
Total reviews added for Hello Lighthouse: 59
T